# Optimize with Hyperactive
[Home](https://simonblanke.github.io/hyperactive-documentation/)

## Install


In [ ]:
!pip install hyperactive

## Sample strategy

In [1]:
import talib.abstract as ta

from lettrade import DataFeed, Strategy, crossover, crossunder
from lettrade.exchange.backtest import ForexBackTestAccount, let_backtest


class SmaCross(Strategy):
    ema1_period = 9
    ema2_period = 21

    def indicators(self, df: DataFeed):
        df["ema1"] = ta.EMA(df, timeperiod=self.ema1_period)
        df["ema2"] = ta.EMA(df, timeperiod=self.ema2_period)

        df["signal_ema_crossover"] = crossover(df.ema1, df.ema2)
        df["signal_ema_crossunder"] = crossunder(df.ema1, df.ema2)

    def next(self, df: DataFeed):
        if len(self.orders) > 0 or len(self.trades) > 0:
            return

        if df.signal_ema_crossover[-1]:
            price = self.data.close[-1]
            self.buy(size=0.1, sl=price - 0.001, tp=price + 0.001)
        elif df.signal_ema_crossunder[-1]:
            price = self.data.close[-1]
            self.sell(size=0.1, sl=price + 0.001, tp=price - 0.001)


lt = let_backtest(
    strategy=SmaCross,
    datas="example/data/data/EURUSD_5m_0_10000.csv",
    account=ForexBackTestAccount,
)

### Example

In [2]:
from hyperactive import Hyperactive
from hyperactive.optimizers import HillClimbingOptimizer


# define the model in a function
def params_parser(args):
    params = [
        ("ema1_period", int(args["ema1_period"])),
        ("ema2_period", int(args["ema2_period"])),
    ]
    return params


def result_parser(result):
    return result["Equity [$]"]


model = lt.optimize_model(
    params_parser=params_parser,
    result_parser=result_parser,
    fork_data=True,
)

# search space determines the ranges of parameters you want the optimizer to search through
search_space = {
    "ema1_period": list(range(5, 25, 1)),
    "ema2_period": list(range(10, 50)),
}

optimizer = HillClimbingOptimizer(epsilon=0.1, distribution="laplace", n_neighbours=4)

# start the optimization run
hyper = Hyperactive()
hyper.add_search(model, search_space, optimizer=optimizer, n_iter=1000)
hyper.run()

[0] _optimize_model (Hill Climbing): 100%|──────────| 1000/1000 [04:51<00:00,  3.43it/s, best_iter=0, best_pos=[9 0], best_score=1183.9] 



Results: '_optimize_model'  
   Best score: 1183.9  
   Best parameter set:
      'ema1_period' : 14.0  
      'ema2_period' : 10.0  
   Best iteration: 0  
 
   Random seed: 918392602  
 
   Evaluation time   : 290.315381526947 sec    [99.96 %]
   Optimization time : 0.10812616348266602 sec    [0.04 %]
   Iteration time    : 290.4235076904297 sec    [3.44 iter/sec]
 


In [3]:
df = hyper.search_data(model)
df

,ema1_period,ema2_period,score
0,18,31,991.4
1,5,13,930.1
2,11,23,851.7
3,11,36,940.7
4,17,23,959.7
...,...,...,...
995,14,12,1060.4
996,5,12,1010.7
997,13,12,1150.3
998,14,11,1170.5


## Plot

### Type 1

In [4]:
import plotly.express as px

fig = px.density_heatmap(
    df,
    x="ema1_period",
    y="ema2_period",
    z="score",
    nbinsx=30,
    nbinsy=30,
    histfunc="max",
    color_continuous_scale="Viridis",
)
fig.show()

# Type 2

In [5]:
import plotly.express as px

fig = px.density_contour(
    df,
    x="ema1_period",
    y="ema2_period",
    z="score",
    histfunc="max",
)
fig.update_traces(contours_coloring="fill", contours_showlabels=True)
fig.show()